In [1]:
from pandas import read_csv, DataFrame
import pandas as pd
import os
import re #Both patterns and strings to be searched can be Unicode strings as well as 8-bit strings.
import math
import cobra
import cobra.test
from __future__ import print_function
from os.path import join
from cobra.io import write_sbml_model
import itertools
from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"

In [2]:
import cobra
from os.path import join
    
# Load a cobra model from anywhere you like
data_dir = "/Users/lizrad/Documents/Vibrio_folder/"
model=cobra.io.read_sbml_model(join(data_dir, "iVnat_newbigg.xml"))



### Upload ecoli model and check metabolite and reaction IDs

In [3]:
data_dir = "/Users/lizrad/Documents/Vibrio_folder/"
ecoli_model=cobra.io.read_sbml_model(join(data_dir, "iML1515.xml"))
ecoli_model

Name,iML1515
Memory address,0x0113ebd080
Number of metabolites,1877
Number of reactions,2712
Objective expression,-1.0*BIOMASS_Ec_iML1515_core_75p37M_reverse_35685 + 1.0*BIOMASS_Ec_iML1515_core_75p37M
Compartments,"cytosol, extracellular space, periplasm"


In [ ]:
count=0
for e_met in ecoli_model.metabolites:
    for met in model.metabolites:
        bigg, compartment=e_met.id.rsplit("_", 1)
        if bigg in met.annotation.values():
            
            
            print(e_met.id, met.id, met.annotation["bigg.metabolite"])
            met.id=bigg+ "_"+ met.compartment[0]
            print("####new ID:",met.id)
            model.repair()
            
        elif bigg in met.annotation.values() and type(met.annotation["bigg.metabolite"])==list:
            print("lista")
            count+=1
count

In [ ]:
count_r=0
for e_reac in ecoli_model.reactions:
    for reac in model.reactions:
        #bigg, compartment=e_reac.id.rsplit("_", 1)
        if e_reac.id in reac.annotation.values():
            
            
            print(e_reac.id, reac.id, reac.annotation["bigg.reaction"])
            reac.id=e_reac.id
            print("####new ID:",reac.id)
            model.repair()
            
        elif bigg in reac.annotation.values() and type(reac.annotation["bigg.reaction"])==list:
            print("lista")
            count_r+=1
count_r

In [ ]:
#not used
for e_met in ecoli_model.metabolites:
    for met in model.metabolites:
        bigg, compartment=e_met.id.rsplit("_", 1)
        if bigg in met.annotation.values():
            print(e_met.id, met.id, met.annotation)
            met.id=bigg+ "_"+ met.compartment[0]
            print("####new ID:",met.id)
            model.repair()


In [ ]:
#not used
for e_reac in ecoli_model.reactions:
    for reac in model.reactions:
        if e_reac.id in reac.annotation.values():
            if "bigg.reaction" in reac.annotation.keys():
                print(e_reac.id, reac.id, reac.annotation)
               # print(e_reac.id, reac.annotation.values() )
               # for value in reac.annotation.values():
                reac.id=e_reac.id
                model.repair()


### Exchange reactions 

In [ ]:
not_found=[]
ex_met_list=[]
for ex in model.exchanges:
   # if ex.id.startswith("EX"):
    pre, seed=ex.id.split("_")
    for met in model.metabolites:
        if seed in met.annotation.values() and "bigg.metabolite" in met.annotation.keys():
            try:
                if type(met.annotation["bigg.metabolite"]) == list:
                    ex_met_list.append(ex)

                    for bigg in ecoli_model.metabolites:
                        identif, compartment= bigg.id.rsplit("_", 1)
                        if identif in met.annotation["bigg.metabolite"]:
                            print("from list", identif, met.annotation["bigg.metabolite"])
                            ex.id="EX_"+identif+"_"+ met.compartment[0]
                            print("From list:", ex.id)
                            model.repair()
                        else:
                            not_found.append(ex.id)


                else:
                    met.annotation["bigg.metabolite"]
                    ex.id="EX_"+met.annotation["bigg.metabolite"]+"_"+ met.compartment[0]
                    model.repair()
                    print(ex.id)
            except:
                pass

                

                
mult_bigg=set(ex_met_list)
mult_bigg


In [ ]:
#write_sbml_model(model, "/Users/lizrad/Documents/Vibrio_folder/iVnat_newbigg.xml")

In [8]:
# generate a json file to load into Escher
cobra.io.save_json_model(model, 'iVnat_newbigg.json')


In [9]:
from escher import Builder
b = Builder(map_name='e_coli_core.Core metabolism')
b.display_in_browser()


Note: You must interrupt the kernel to end this command

Serving to http://127.0.0.1:7655/
[Ctrl-C to exit from terminal, or Ctrl-M i i to interrupt notebook kernel]


127.0.0.1 - - [11/Sep/2018 14:14:21] "GET / HTTP/1.1" 200 -



stopping Server...


In [ ]:
maps = Builder(model=model, map_name="iJO1366.Central metabolism")

maps.display_in_browser()

In [ ]:
maps = Builder(model=model, map_name="iJO1366.Central metabolism")

maps.display_in_browser()



In [ ]:
for x in model.metabolites.fdp_c.reactions:
    print(x.reaction, x.name, x.id)

In [ ]:
model.reactions.MNXR106666.annotation

In [ ]:
count=0
count_list=0
for met in model.metabolites:
    if "chebi" in met.annotation.keys():
        print(met.id, met.annotation["chebi"])
        count+=1
        if type(met.annotation["chebi"])==list:
            count_list+=1
            
count
len(model.metabolites)
count_list

In [ ]:
rcount=0
rcount_list=0
for reac in model.reactions:
    if "chebi" in reac.annotation.keys():
        print(reac.id, reac.annotation["chebi"])
        rcount+=1
        if type(reac.annotation["chebi"])==list:
            rcount_list+=1
            
rcount
len(model.reactions)
rcount_list

In [ ]:

model.metabolites.C3_c.annotation

In [ ]:
modelseed = read_csv("/Users/lizrad/Documents/Vibrio_folder/compound_aliases_modelseed.tsv" , sep='\t' , low_memory=False)
modelseed

In [ ]:
import numpy as np
df_seed = modelseed.loc[36463:39317, "MS ID":'External ID']
pivoted_seed = df_seed.pivot_table( columns='MS ID',values='External ID', aggfunc=list)

pivoted_seed
modelseed_met_dict = pivoted_seed.to_dict('index')
modelseed_met_dict2 = modelseed_met_dict['External ID']
modelseed_met_dict2
modelseed_met_dict2["cpd00013"]


In [4]:
modelseed_reac = read_csv("/Users/lizrad/Documents/Vibrio_folder/reaction_aliases_modelseed.tsv" , sep='\t' , low_memory=False)
modelseed_reac

,MS ID,Old MS ID,External ID,Source
0,rxn07912,rxn07912,12DGR120tipp,BiGG
1,rxn07913,rxn07913,12DGR140tipp,BiGG
2,rxn07914,rxn07914,12DGR141tipp,BiGG
3,rxn07915,rxn07915,12DGR160tipp,BiGG
4,rxn07916,rxn07916,12DGR161tipp,BiGG
5,rxn07917,rxn07917,12DGR180tipp,BiGG
6,rxn07918,rxn07918,12DGR181tipp,BiGG
7,rxn07919,rxn07919,12PPDRtex,BiGG
8,rxn07920,rxn07920,12PPDRtpp,BiGG
9,rxn07921,rxn07921,12PPDStex,BiGG


In [5]:
import numpy as np
df_seed = modelseed_reac.loc[1:6806, "MS ID":'External ID']
pivoted_seed = df_seed.pivot_table(columns='MS ID',values='External ID',aggfunc = list)

pivoted_seed
modelseed_reac_dict = pivoted_seed.to_dict('index')
modelseed_reac_dict2 = modelseed_reac_dict['External ID']
modelseed_reac_dict2

MS ID,rxn00001,rxn00002,rxn00006,rxn00007,rxn00010,rxn00014,rxn00016,rxn00018,rxn00029,rxn00044,...,rxn39083,rxn39084,rxn39085,rxn39086,rxn39087,rxn39088,rxn39089,rxn39090,rxn39091,rxn39092
External ID,"[IPP1, PPA, PPA_1]",[DUR1_2],"[CAT, CTA1, CTT1]","[ATH1, NTH1, NTH2, TREH]","[GLXCBL, GLXCL]","[CCP, CYCPO]",[HXAD],[RBPC],"[HEM2, PPBNGS]",[AHXDH],...,[G16MTg],[GCC2bim],[NH4tm],[NTRLASE2],[PIN4Kn_SC],[PSERDv_SC],[PSERSm_SC],[SBPP2er],[THMDPe],[SO3De]


{'rxn00001': ['IPP1', 'PPA', 'PPA_1'],
 'rxn00002': ['DUR1_2'],
 'rxn00006': ['CAT', 'CTA1', 'CTT1'],
 'rxn00007': ['ATH1', 'NTH1', 'NTH2', 'TREH'],
 'rxn00010': ['GLXCBL', 'GLXCL'],
 'rxn00014': ['CCP', 'CYCPO'],
 'rxn00016': ['HXAD'],
 'rxn00018': ['RBPC'],
 'rxn00029': ['HEM2', 'PPBNGS'],
 'rxn00044': ['AHXDH'],
 'rxn00060': ['HEM3'],
 'rxn00061': ['ATPH1'],
 'rxn00062': ['ATPM', 'ATPOBJ', 'NTP1'],
 'rxn00063': ['NTPP6'],
 'rxn00065': ['ADNCYC', 'CYR1'],
 'rxn00069': ['GLT1', 'GLUSx'],
 'rxn00075': ['NADNm'],
 'rxn00076': ['NADDP', 'NADDPe'],
 'rxn00077': ['NADK', 'POS5_1', 'UTR1_1', 'YEL041W_1'],
 'rxn00083': ['NADTRHD', 'TEST_NADTRHD', 'THD1', 'THD5'],
 'rxn00085': ['GLUSy'],
 'rxn00086': ['GLR1', 'GTHO', 'GTHOr', 'GTHRD'],
 'rxn00088': ['NADPPPS', 'U101'],
 'rxn00092': ['NDP1'],
 'rxn00095': ['AP4AH', 'AP4AHr'],
 'rxn00096': ['APA2', 'ATPATF1'],
 'rxn00097': ['ADK1', 'ADK1_1'],
 'rxn00100': ['DPCOAK', 'U96'],
 'rxn00101': ['UREA2'],
 'rxn00102': ['H2CO3D', 'HCO3E'],
 'rxn00104': 

In [6]:
for met in model.metabolites: 
     if met.annotation["seed.compound"] in modelseed_met_dict2.keys():
        #print("modelseed bigg id", modelseed_met_dict2[met.annotation["seed.compound"]])
        if "bigg.metabolite" in met.annotation and met.annotation["bigg.metabolite"] in modelseed_met_dict2[met.annotation["seed.compound"]]:
            print("bigg annotation", met.annotation["bigg.metabolite"])
        else:
            print(met.id, met.annotation, modelseed_met_dict2[met.annotation["seed.compound"]])

NameError: name 'modelseed_met_dict2' is not defined

In [7]:
new_bigg_ann_dict={}

for reac in model.reactions:

    if "seed.compound" in reac.annotation and reac.annotation["seed.compound"] in modelseed_reac_dict2.keys():
        #print("modelseed bigg id", modelseed_reac_dict2[reac.annotation["seed.compound"]])
        if "bigg.reaction" in reac.annotation and reac.annotation["bigg.reaction"] in modelseed_reac_dict2[reac.annotation["seed.compound"]]:
            
            new_bigg_ann_dict[reac.annotation["seed.compound"]]={}
            new_bigg_ann_dict[reac.annotation["seed.compound"]]["bigg_ann"]=reac.annotation["bigg.reaction"]
            new_bigg_ann_dict[reac.annotation["seed.compound"]]["in_modelseed"]=modelseed_reac_dict2[reac.annotation["seed.compound"]]
            both_db_ann=[]
            if type(reac.annotation["bigg.reaction"])==list:
                for rl in reac.annotation["bigg.reaction"]:
                    both_db_ann.append(rl)
            else:
                both_db_ann.append(reac.annotation["bigg.reaction"])
                
            if type(modelseed_reac_dict2[reac.annotation["seed.compound"]])==list:
                for rl in modelseed_reac_dict2[reac.annotation["seed.compound"]]:
                    both_db_ann.append(rl)
            else:
                both_db_ann.append(modelseed_reac_dict2[reac.annotation["seed.compound"]])
                
            new_bigg_ann_dict[reac.annotation["seed.compound"]]["both"]=set(both_db_ann)
            #there is annotation not list and  it is the same as in modelseed db
        elif "bigg.reaction" in reac.annotation and type(reac.annotation["bigg.reaction"])==list:
            #annotation is list but check if is the same
            for reac_list in reac.annotation["bigg.reaction"]: 
                if reac_list in modelseed_reac_dict2[reac.annotation["seed.compound"]]:
                    
                    new_bigg_ann_dict[reac.annotation["seed.compound"]]={}
                    new_bigg_ann_dict[reac.annotation["seed.compound"]]["bigg_ann"]=reac.annotation["bigg.reaction"]
                    new_bigg_ann_dict[reac.annotation["seed.compound"]]["in_modelseed"]=modelseed_reac_dict2[reac.annotation["seed.compound"]]
                    
                    both_db_ann=[]
                    if type(reac.annotation["bigg.reaction"])==list:
                        for rl in reac.annotation["bigg.reaction"]:
                            both_db_ann.append(rl)
                    else:
                        both_db_ann.append(reac.annotation["bigg.reaction"])

                    if type(modelseed_reac_dict2[reac.annotation["seed.compound"]])==list:
                        for rl in modelseed_reac_dict2[reac.annotation["seed.compound"]]:
                            both_db_ann.append(rl)
                    else:
                        both_db_ann.append(modelseed_reac_dict2[reac.annotation["seed.compound"]])

                    new_bigg_ann_dict[reac.annotation["seed.compound"]]["both"]=set(both_db_ann)

        else:
            #bigg not in annotation, but there is in the modelseed
       
            new_bigg_ann_dict[reac.annotation["seed.compound"]]={}
            new_bigg_ann_dict[reac.annotation["seed.compound"]]["bigg_ann"]="MISSING"
            new_bigg_ann_dict[reac.annotation["seed.compound"]]["in_modelseed"]=modelseed_reac_dict2[reac.annotation["seed.compound"]]
            
            newid=modelseed_reac_dict2[reac.annotation["seed.compound"]][0].replace(" ", "")
        
            reac.id=newid
            reac.id
            model.repair()
new_bigg_ann_dict

'FAS1_6_c'

'HEPTT'

'PDX5PS2'

'AMPEP13'

'PFK'

...

'KDOPP'

'DHNAOT4'

'RHAT1'

'MNt2'

{'rxn02201': {'bigg_ann': ['DHPS2', 'DHPS2_1', 'FOLD3', 'FOLD3_1', 'FOLD3m'],
  'in_modelseed': ['DHPS2', 'DHPS2FOLD3', 'FOLD3', 'FOLDm'],
  'both': {'DHPS2',
   'DHPS2FOLD3',
   'DHPS2_1',
   'FOLD3',
   'FOLD3_1',
   'FOLD3m',
   'FOLDm'}},
 'rxn00351': {'bigg_ann': ['GTHS', 'GTHS_1'],
  'in_modelseed': ['GSH2', 'GTHS'],
  'both': {'GSH2', 'GTHS', 'GTHS_1'}},
 'rxn00836': {'bigg_ann': 'HXPRT',
  'in_modelseed': ['HPT1_1', 'HXPRT', 'HXPRTr'],
  'both': {'HPT1_1', 'HXPRT', 'HXPRTr'}},
 'rxn02209': {'bigg_ann': ['ALCD21_D', 'LCARR'],
  'in_modelseed': ['LCARR'],
  'both': {'ALCD21_D', 'LCARR'}},
 'rxn08180': {'bigg_ann': 'BTS4', 'in_modelseed': ['BTS4'], 'both': {'BTS4'}},
 'rxn05318': {'bigg_ann': ['ADNt2', 'ADNt2pp'],
  'in_modelseed': ['ADNt2', 'ADNt2r', 'U174', 'U185', 'U191'],
  'both': {'ADNt2', 'ADNt2pp', 'ADNt2r', 'U174', 'U185', 'U191'}},
 'rxn00062': {'bigg_ann': ['ATPM', 'ATPPHm', 'ATPS', 'NTP1'],
  'in_modelseed': ['ATPM', 'ATPOBJ', 'NTP1'],
  'both': {'ATPM', 'ATPOBJ', 'ATP

In [ ]:
model.reactions.DHNAOT4
ecoli_model.reactions.DHNAOT4

In [ ]:
for eco in ecoli_model.reactions:

    for d in new_bigg_ann_dict.keys():
        if eco.id in new_bigg_ann_dict[d] and eco.id not in new_bigg_ann_dict[d]["bigg_ann"]:
            print(eco.id, new_bigg_ann_dict[d])